# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества F1 не меньше 0.75.

План по выполнению проекта

1. Загрузим и подготовим данные.
2. Обучим разные модели.
3. Сделаем выводы.


Описание данных

Данные находятся в файле toxic_comments.csv. Столбец text в нём содержит текст комментария, а toxic — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


import pandas as pd 
import numpy as np 
from sklearn.pipeline import Pipeline
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import time
import warnings
warnings.filterwarnings('ignore')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
import plotly.graph_objects as go

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\79811\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pip install  transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install  torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [6]:
from lightgbm import LGBMClassifier 
from pymystem3 import Mystem
import torch
import transformers

In [7]:
try:
    data = pd.read_csv('C:/Users/79811/Downloads/toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')
data.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [9]:
data.isna().mean()

text     0.0
toxic    0.0
dtype: float64

In [10]:
data.describe(include='all')

,text,toxic
count,159571,159571.000000
unique,159571,NaN
top,Explanation\nWhy the edits made under my usern...,NaN
freq,1,NaN
mean,NaN,0.101679
std,NaN,0.302226
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,0.000000


In [11]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [12]:
data.duplicated().sum()

0

**ВЫВОДЫ ПО ДАННЫМ** 

- 159 571 строк
- Пропусков и дубликатов не замечено 
- только 10% твитов являются токсичными
- в столбце text содержатся тексты  твиты


In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [14]:
%%time
data['text'] = data['text'].map(lambda x: clean_text(x))

Wall time: 7.46 s


## Обучение

In [15]:
tr = data.drop('toxic', axis = 1)
te = data['toxic']

In [16]:
tr_train, tr_test, te_train, te_test = train_test_split(tr, te, test_size = 0.3, random_state = 12345)

LogisticRegression

In [17]:
%%time

log_reg = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LogisticRegression(random_state=12345))])

params = {'clf__C': [0.1, 1, 10, 100],
          'clf__class_weight': ['balanced', None]}

log_reg_grid = GridSearchCV(estimator=log_reg, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
log_reg_grid.fit(tr_train['text'], te_train)
log_reg_params_best = log_reg_grid.best_params_

print(log_reg_params_best)
print(log_reg_grid.best_score_)


{'clf__C': 10, 'clf__class_weight': 'balanced'}
0.7737116850856252
Wall time: 6min 38s


LGBMClassifier

In [18]:
%%time

lgbm_ = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)),
    ('clf', LGBMClassifier(random_state=12345))])

params = {
  'clf__n_estimators': [200],
  'clf__learning_rate': [0.15, 0.25],
  'clf__max_depth': [8, 10, -1]}

lgbm_grid = GridSearchCV(estimator=lgbm_, param_grid=params, cv=3, scoring='f1', n_jobs=-1, refit=False)
lgbm_grid.fit(tr_train['text'],te_train)
lgbm_params_best = lgbm_grid.best_params_

print(lgbm_params_best)
print(lgbm_grid.best_score_)



{'clf__learning_rate': 0.25, 'clf__max_depth': -1, 'clf__n_estimators': 200}
0.7675168724246589
Wall time: 13min 8s


In [19]:
vectorize = TfidfVectorizer(ngram_range=(1,3),
               min_df=3, max_df=0.9, use_idf=1,
               smooth_idf=1, sublinear_tf=1, stop_words=stopwords)

In [20]:
tr_train = vectorize.fit_transform(tr_train['text'])
tr_test = vectorize.transform(tr_test['text'])

In [21]:
log_reg_mod = LogisticRegression(C=10, class_weight='balanced', random_state=12345)
log_reg_mod.fit(tr_train, te_train)

LogisticRegression(C=10, class_weight='balanced', random_state=12345)

In [22]:
lgbm_mod = LGBMClassifier(learning_rate=0.25, max_depth=-1, n_estimators=200, random_state = 12345)
lgbm_mod.fit(tr_train, te_train)

LGBMClassifier(learning_rate=0.25, n_estimators=200, random_state=12345)

In [23]:
def get_f1(fitted_model):
    test_pred = fitted_model.predict(tr_test)
    test_f1 = f1_score(te_test, test_pred)
    
    print('F1 on test: {:.3f}'.format(test_f1))

In [24]:
get_f1(log_reg_mod)

F1 on test: 0.783


In [25]:
get_f1(lgbm_mod)

F1 on test: 0.777


## BERT

In [26]:
pip install pytorch_pretrained_bert

In [27]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [28]:
import random

In [38]:
try:
    df = pd.read_csv('C:/Users/79811/Downloads/toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')
df.head(10)
df = df.sample(500, random_state = 12345).reset_index(drop=True)

tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True, do_basic_tokenize=True)
model = transformers.BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, #текст
                                                         add_special_tokens=True, #особые токены начала и конца текста
                                                         truncation=True)) #усечение до числа входов модели
)

In [40]:
tokenized.map(lambda x: len(x)).max()

512

In [41]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [42]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/5 [00:00<?, ?it/s]

In [43]:
len(df['toxic']) 

500

In [44]:
print(len(embeddings)) 

5


In [45]:
features = np.concatenate(embeddings)
target = df['toxic']

In [46]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=12345) #тут немного переиграем, для подбора порога используем
#отдельную выборку, валидационную
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test, test_size=0.5, random_state=12345)
def score(model):
    model.fit(features_train, target_train)
    predictions = model.predict(features_test)
    f1 = f1_score(target_test, predictions)
    return f1

In [47]:
model = LogisticRegression(class_weight='balanced', random_state=12345)
print(score(model))
model=LGBMClassifier(class_weight='balanced', random_state=12345)
print(score(model))

0.4615384615384615
0.25


In [48]:
from sklearn.linear_model import LogisticRegressionCV #
c_values = np.logspace(-2, 3, 500) #значения C для перебора
model = LogisticRegressionCV(Cs=c_values, verbose=1, n_jobs=-1, penalty='l1', scoring='f1', class_weight='balanced', solver='liblinear').fit(features_train, target_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   23.5s remaining:   35.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   26.4s finished


In [49]:
best_tr = None #лучшего порога для логистической регрессии сначала нет, как будем находить лучший вариант, будем его сюда перезаписывать
best_f1 = 0 #примем лучшее значение f1 изначально за самый плохой вариант, по мере того, как будем находить лучший вариант, будем перезаписывать
for tr in (np.array(range(0, 100)) * 0.01): #теперь для настроенной версии перебираем порог
    print(
        f1_score((model.predict_proba(features_valid)[:,1] > tr) * 1.0, target_valid), #f1 с текщим порогом, заметь, что проверяем на валидационной выборке
         tr #сам порог
          )
    if f1_score((model.predict_proba(features_valid)[:,1] > tr) * 1.0, target_valid) > best_f1: #если текущее f1 лучше того, который сейчас записан как лучший
        best_f1 = f1_score((model.predict_proba(features_valid)[:,1] > tr) * 1.0, target_valid) #перезапишем лучшее f1 текущим значением
        best_tr = tr #и возьмём текущий порог как лучший

0.17073170731707318 0.0
0.6666666666666665 0.01
0.6250000000000001 0.02
0.6666666666666666 0.03
0.7142857142857143 0.04
0.7142857142857143 0.05
0.7142857142857143 0.06
0.7142857142857143 0.07
0.7142857142857143 0.08
0.7142857142857143 0.09
0.7142857142857143 0.1
0.7142857142857143 0.11
0.7142857142857143 0.12
0.7142857142857143 0.13
0.7142857142857143 0.14
0.7142857142857143 0.15
0.7142857142857143 0.16
0.7142857142857143 0.17
0.7142857142857143 0.18
0.7142857142857143 0.19
0.7142857142857143 0.2
0.7142857142857143 0.21
0.7142857142857143 0.22
0.7142857142857143 0.23
0.7142857142857143 0.24
0.7142857142857143 0.25
0.7142857142857143 0.26
0.7142857142857143 0.27
0.7142857142857143 0.28
0.7142857142857143 0.29
0.7142857142857143 0.3
0.7142857142857143 0.31
0.7142857142857143 0.32
0.7142857142857143 0.33
0.7142857142857143 0.34
0.7142857142857143 0.35000000000000003
0.7142857142857143 0.36
0.6153846153846153 0.37
0.6153846153846153 0.38
0.6153846153846153 0.39
0.6153846153846153 0.4
0.615

In [50]:
print(f1_score((model.predict_proba(features_valid)[:,1] > best_tr) * 1.0, target_valid)) #ещё раз смотрим f1 на валидации, выглядит не достаточно хорошо

0.7272727272727273


In [51]:
print(f1_score( #f1 на тесте
    (model.fit(np.concatenate((features_train, features_valid)), np.concatenate((target_train.values, target_valid.values))).predict_proba(features_test)[:,1] > best_tr) * 1.0,
    #обучим модель на объединенной валидационной выборке и обучающей, не пропадать же данным, чем больше данных, тем качественнее можно получить предсказание в общем случае
    #и получим для неё предсказания с учётом найденного ранее порога
     target_test))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   25.9s remaining:   38.9s


0.25


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   33.3s finished


## Выводы

In [52]:
results = {
    'Model' : ['LGBMRegressor', 'LogisticRegression', 'BERT_LogisticRegression', 'BERT_LGBMRegressor'],
    'F1' :pd.Series([0.777,  0.783, 0.425, 0.25]),
    }
display(pd.DataFrame(results))

,Model,F1
0,LGBMRegressor,0.777
1,LogisticRegression,0.783
2,BERT_LogisticRegression,0.425
3,BERT_LGBMRegressor,0.250


В общем лучший результат у меня получился на LogisticRegression. Я так же попытался с бертом побороться, но вышел не очень удовльтворительный результат. Сначала у меня выдавало 0.678 результат для LogisticRegression, а теперь 0.425. 
